# Sistema de Homologación de Medicamentos - Entrenamiento

Notebook para entrenar modelos de clustering jerárquico especializado en homologación de medicamentos con scoring personalizado.

## Importación de Librerías y Configuración

Librerías necesarias para clustering, métricas de evaluación y procesamiento de datos con Polars.

In [1]:

"""
🎯 SISTEMA DE HOMOLOGACIÓN DE MEDICAMENTOS - PASO 2
===================================================
Implementación de Clustering Jerárquico para Homologación de Medicamentos
"""

import polars as pl
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings('ignore')


## Clase HomologacionClusteringModel

Implementación del modelo de clustering especializado que combina clustering primario por variables críticas (ATC+VÍA) con clustering secundario por similitud.

In [2]:
class HomologacionClusteringModel:
    """
    Modelo de clustering especializado para homologación de medicamentos.
    
    Estrategia:
    1. Clustering primario por ATC+VÍA (variables críticas)
    2. Clustering secundario por características específicas
    3. Sistema de similitud con pesos jerárquicos
    
    Examples:
        >>> model = HomologacionClusteringModel()
        >>> model.fit(df_training)
        >>> homologos = model.recomendar_homologos('10045-1', n_recomendaciones=5)
    """
    
    def __init__(self, pesos_jerarquicos: dict = None):
        """
        Inicializa el modelo de clustering para homologación.
        
        Args:
            pesos_jerarquicos (dict): Pesos para variables críticas vs importantes
        """
        self.pesos_jerarquicos = {
            'ATC': 0.40,                    # 40% - CRÍTICO (mantener alto)
            'VIA_ADMINISTRACION': 0.30,     # 30% - CRÍTICO (aumentado)
            'FORMA_FARMACEUTICA': 0.20,     # 20% - IMPORTANTE 
            'CANTIDAD_SIMILITUD': 0.10,     # 10% - IMPORTANTE
            'PRINCIPIO_BONUS': 0.00         # 0% base + bonus variable
        }
        
        # Modelos de clustering
        self.cluster_primario = None
        self.cluster_secundario = None
        self.scaler = StandardScaler()
        self.knn_model = None
        
        # Datos de entrenamiento
        self.df_validos = None
        self.df_referencia = None
        self.features_clustering = None
        self.datos_escalados = None
        
        # Mapeos y vocabularios
        self.combo_clusters = {}
        self.cluster_info = {}
        
    def preparar_features_clustering(self, df: pl.DataFrame) -> tuple:
        """
        Prepara las features optimizadas para clustering de homologación.
        
        Args:
            df (pl.DataFrame): Dataset de entrenamiento preparado
            
        Returns:
            tuple: (features_criticas, features_importantes, features_combinadas)
            
        Notes:
            Separa features críticas de importantes para aplicar pesos diferentes
        """
        print("🔧 PREPARANDO FEATURES PARA CLUSTERING")
        print("=" * 45)
        
        # FEATURES CRÍTICAS (deben coincidir exactamente para homologación)
        features_criticas = [
            'ATC_label',
            'VÍA ADMINISTRACIÓN_label', 
            'PRINCIPIO ACTIVO_label'
        ]
        
        # FEATURES IMPORTANTES (permiten variación con penalización)
        features_importantes = [
            'FORMA FARMACÉUTICA_label',
            'CANTIDAD_log',
            'CANTIDAD_CUM_log',
            'RATIO_CANTIDAD',
            'CANTIDAD_bin',
            'UNIDAD MEDIDA_label'
        ]
        
        # FEATURES DE PROBABILIDAD (para scoring)
        features_probabilidad = [
            'ATC_prob_validos',
            'VÍA ADMINISTRACIÓN_prob_validos',
            'PRINCIPIO ACTIVO_prob_validos',
            'score_prob_critica'
        ]
        
        # Verificar disponibilidad
        features_criticas_disp = [f for f in features_criticas if f in df.columns]
        features_importantes_disp = [f for f in features_importantes if f in df.columns]
        features_prob_disp = [f for f in features_probabilidad if f in df.columns]
        
        print(f"✅ Features críticas: {len(features_criticas_disp)}")
        print(f"✅ Features importantes: {len(features_importantes_disp)}")
        print(f"✅ Features probabilidad: {len(features_prob_disp)}")
        
        # Combinar todas las features
        todas_features = features_criticas_disp + features_importantes_disp + features_prob_disp
        
        return features_criticas_disp, features_importantes_disp, todas_features
    
    def clustering_por_combo_critico(self, df_validos: pl.DataFrame) -> dict:
        """
        Realiza clustering primario agrupando por combinaciones críticas ATC+VÍA.
        
        Args:
            df_validos (pl.DataFrame): Dataset de medicamentos válidos
            
        Returns:
            dict: Mapeo de combinaciones ATC+VÍA a clusters
            
        Notes:
            Este clustering asegura que medicamentos con diferentes ATC+VÍA 
            nunca se recomienden como homólogos
        """
        print("🎯 CLUSTERING PRIMARIO: Agrupación por ATC+VÍA")
        print("=" * 50)
        
        # Analizar combinaciones ATC+VÍA
        combos_info = (df_validos
                      .group_by('ATC_VIA_combo')
                      .agg([
                          pl.len().alias('count'),
                          pl.col('ATC').first().alias('ATC_ejemplo'),
                          pl.col('VÍA ADMINISTRACIÓN').first().alias('VIA_ejemplo'),
                          pl.col('PRINCIPIO ACTIVO').n_unique().alias('principios_unicos')
                      ])
                      .sort('count', descending=True))
        
        print(f"📊 Total combinaciones ATC+VÍA encontradas: {combos_info.height:,}")
        
        # Asignar cluster_id a cada combinación
        combo_clusters = {}
        cluster_info = {}
        
        for i in range(combos_info.height):
            row = combos_info.row(i)
            combo = row[0]
            count = row[1]
            atc = row[2]
            via = row[3]
            principios = row[4]
            
            combo_clusters[combo] = i
            cluster_info[i] = {
                'combo': combo,
                'count': count,
                'atc': atc,
                'via': via,
                'principios_unicos': principios,
                'cluster_id': i
            }
        
        # Mostrar top clusters
        print(f"\n🏆 TOP 10 CLUSTERS MÁS GRANDES:")
        for i in range(min(10, combos_info.height)):
            info = cluster_info[i]
            print(f"   {i+1:2d}. {info['atc']} + {info['via']}: {info['count']:,} medicamentos ({info['principios_unicos']} principios)")
        
        return combo_clusters, cluster_info
    
    def clustering_secundario_por_similitud(self, df_validos: pl.DataFrame, features_importantes: list) -> None:
        """
        Aplica clustering secundario dentro de cada grupo ATC+VÍA.
        
        Args:
            df_validos (pl.DataFrame): Dataset de medicamentos válidos
            features_importantes (list): Features para clustering secundario
            
        Notes:
            Este clustering agrupa medicamentos similares dentro del mismo ATC+VÍA
        """
        print("🔍 CLUSTERING SECUNDARIO: Similitud dentro de grupos")
        print("=" * 55)
        
        # Preparar datos para clustering secundario
        features_disponibles = [f for f in features_importantes if f in df_validos.columns]
        
        # Extraer datos numéricos para clustering (convertir a numpy para sklearn)
        datos_clustering = df_validos.select(features_disponibles).to_numpy()
        
        # Escalar datos
        datos_escalados = self.scaler.fit_transform(datos_clustering)
        
        # Determinar número óptimo de clusters usando método del codo
        max_k = min(50, len(datos_escalados) // 10)  # Máximo 50 clusters o 1 cada 10 medicamentos
        
        if max_k >= 2:
            print(f"🔍 Buscando k óptimo entre 2 y {max_k} clusters...")
            
            # Probar diferentes valores de k
            inertias = []
            k_range = range(2, min(max_k + 1, 21))  # Limitar a máximo 20 para eficiencia
            
            for k in k_range:
                kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                kmeans.fit(datos_escalados)
                inertias.append(kmeans.inertia_)
            
            # Elegir k usando método del codo simplificado
            if len(inertias) > 2:
                # Calcular second differences para encontrar el "codo"
                diffs = np.diff(inertias)
                second_diffs = np.diff(diffs)
                k_optimo = k_range[np.argmin(second_diffs) + 1]
            else:
                k_optimo = k_range[0]
            
            print(f"✅ K óptimo seleccionado: {k_optimo}")
            
            # Entrenar modelo final
            self.cluster_secundario = KMeans(n_clusters=k_optimo, random_state=42, n_init=10)
            clusters_secundarios = self.cluster_secundario.fit_predict(datos_escalados)
            
            # Evaluar calidad si hay suficientes datos
            if len(datos_escalados) > k_optimo:
                silhouette = silhouette_score(datos_escalados, clusters_secundarios)
                print(f"📊 Silhouette Score: {silhouette:.3f}")
            
        else:
            print("⚠️  Pocos datos para clustering secundario, usando cluster único")
            clusters_secundarios = np.zeros(len(datos_escalados))
            k_optimo = 1
        
        # Entrenar modelo KNN para búsqueda de similares
        self.knn_model = NearestNeighbors(n_neighbors=min(20, len(datos_escalados)), metric='cosine')
        self.knn_model.fit(datos_escalados)
        
        # Guardar datos para uso posterior
        self.datos_escalados = datos_escalados
        self.features_clustering = features_disponibles
        
        print(f"✅ Clustering secundario completado: {k_optimo} clusters")
        print(f"✅ Modelo KNN entrenado para búsqueda de similares")
    
    def fit(self, df_training: pl.DataFrame) -> None:
        """
        Entrena el modelo completo de clustering para homologación.
        
        Args:
            df_training (pl.DataFrame): Dataset preparado para entrenamiento
            
        Examples:
            >>> model.fit(df_training)
        """
        print("🚀 ENTRENANDO MODELO DE CLUSTERING PARA HOMOLOGACIÓN")
        print("=" * 65)
        
        # Filtrar solo medicamentos válidos para entrenamiento
        self.df_validos = df_training.filter(pl.col('VALIDO') == 1)
        print(f"📊 Medicamentos válidos para entrenamiento: {self.df_validos.height:,}")
        
        # Preparar features
        features_criticas, features_importantes, todas_features = self.preparar_features_clustering(self.df_validos)
        
        # Clustering primario por combinaciones críticas
        self.combo_clusters, self.cluster_info = self.clustering_por_combo_critico(self.df_validos)
        
        # Clustering secundario por similitud
        self.clustering_secundario_por_similitud(self.df_validos, features_importantes)
        
        # Guardar dataset de referencia (todos los datos para búsqueda)
        self.df_referencia = df_training
        
        print(f"\n✅ MODELO ENTRENADO EXITOSAMENTE")
        print(f"🎯 Clusters primarios (ATC+VÍA): {len(self.combo_clusters):,}")
        print(f"🔍 Modelo KNN listo para búsqueda de similares")
        
    def obtener_info_medicamento(self, cum: str) -> dict:
        """
        Obtiene información completa de un medicamento por su CUM.
        
        Args:
            cum (str): Código CUM del medicamento
            
        Returns:
            dict: Información del medicamento o None si no existe
        """
        medicamento = self.df_referencia.filter(pl.col('CUM') == cum)
        
        if medicamento.height == 0:
            return None
        
        # Convertir a diccionario usando row con named=True
        info = medicamento.to_pandas().iloc[0].to_dict()  # Polars a pandas para facilitar el manejo
        return info

## Entrenamiento del Modelo de Clustering

Carga del dataset procesado y entrenamiento del modelo de clustering jerárquico para homologación de medicamentos.

In [3]:
# Ejecutar entrenamiento del modelo
print("🎯 INICIANDO ENTRENAMIENTO DEL MODELO DE CLUSTERING")
print("=" * 70)

df_training = pl.read_parquet('../data/dataset_entrenamiento_homologacion.parquet')


# Crear y entrenar modelo
modelo_homologacion = HomologacionClusteringModel()
modelo_homologacion.fit(df_training)

print(f"\n🎉 MODELO LISTO PARA HOMOLOGACIÓN")
print(f"📋 Medicamentos de referencia: {modelo_homologacion.df_validos.height:,}")

🎯 INICIANDO ENTRENAMIENTO DEL MODELO DE CLUSTERING
🚀 ENTRENANDO MODELO DE CLUSTERING PARA HOMOLOGACIÓN
📊 Medicamentos válidos para entrenamiento: 62,006
🔧 PREPARANDO FEATURES PARA CLUSTERING
✅ Features críticas: 3
✅ Features importantes: 6
✅ Features probabilidad: 4
🎯 CLUSTERING PRIMARIO: Agrupación por ATC+VÍA
📊 Total combinaciones ATC+VÍA encontradas: 2,119

🏆 TOP 10 CLUSTERS MÁS GRANDES:
    1. N02BE51 + ORAL: 2,540 medicamentos (110 principios)
    2. A07CA99 + ORAL: 1,976 medicamentos (38 principios)
    3. M01AE51 + ORAL: 1,173 medicamentos (50 principios)
    4. A02BC05 + ORAL: 699 medicamentos (53 principios)
    5. B05BA10 + INTRAVENOSA: 679 medicamentos (140 principios)
    6. C10AA07 + ORAL: 631 medicamentos (59 principios)
    7. R01BA53 + ORAL: 601 medicamentos (22 principios)
    8. A11AA01 + ORAL: 596 medicamentos (96 principios)
    9. M01AE01 + ORAL: 557 medicamentos (31 principios)
   10. V03AN01 + INHALACION: 548 medicamentos (14 principios)
🔍 CLUSTERING SECUNDARIO: 

## Sistema de Recomendación de Homólogos

Implementación del sistema de recomendación que utiliza scoring jerárquico con pesos personalizados para encontrar medicamentos homólogos.

In [5]:
"""
🎯 SISTEMA DE HOMOLOGACIÓN DE MEDICAMENTOS - PASO 3
===================================================
Sistema de Recomendación de Medicamentos Homólogos con Scoring Jerárquico
"""

import polars as pl
from typing import Tuple
import warnings
warnings.filterwarnings('ignore')

class SistemaRecomendacionHomologos:
    """
    Sistema completo de recomendación de medicamentos homólogos.
    
    Funcionalidades:
    1. Búsqueda de homólogos por CUM
    2. Scoring jerárquico con pesos personalizados
    3. Explicaciones de motivos de recomendación
    4. Manejo de casos sin homólogos disponibles
    
    Examples:
        >>> sistema = SistemaRecomendacionHomologos(modelo_homologacion)
        >>> recomendaciones = sistema.recomendar_homologos('10045-1', n_recomendaciones=5)
    """
    
    def __init__(self, modelo_clustering):
        """
        Inicializa el sistema de recomendación.
        
        Args:
            modelo_clustering: Modelo de clustering entrenado
        """
        self.modelo = modelo_clustering
        self.pesos = modelo_clustering.pesos_jerarquicos
    
    def calcular_score_similitud(self, medicamento_origen: dict, medicamento_candidato: dict) -> Tuple[float, dict]:
        """
        Calcula score de similitud entre medicamento origen y candidato.
        
        Args:
            medicamento_origen (dict): Datos del medicamento de consulta
            medicamento_candidato (dict): Datos del medicamento candidato
            
        Returns:
            Tuple[float, dict]: (score_total, detalle_scoring)
            
        Notes:
            Aplica pesos jerárquicos: ATC(40%) + VÍA(25%) + PRINCIPIO(20%) + FORMA(10%) + CANTIDAD(5%)
        """
        detalle = {}
        score_total = 0.0
        
        # 1. ATC (40% - CRÍTICO)
        if medicamento_origen['ATC'] == medicamento_candidato['ATC']:
            score_atc = 1.0
            detalle['ATC'] = {'coincide': True, 'score': 1.0, 'peso': self.pesos['ATC']}
        else:
            score_atc = 0.0
            detalle['ATC'] = {'coincide': False, 'score': 0.0, 'peso': self.pesos['ATC']}
        
        score_total += score_atc * self.pesos['ATC']
        
        # 2. VÍA ADMINISTRACIÓN (25% - CRÍTICO)
        if medicamento_origen['VÍA ADMINISTRACIÓN'] == medicamento_candidato['VÍA ADMINISTRACIÓN']:
            score_via = 1.0
            detalle['VIA'] = {'coincide': True, 'score': 1.0, 'peso': self.pesos['VIA_ADMINISTRACION']}
        else:
            score_via = 0.0
            detalle['VIA'] = {'coincide': False, 'score': 0.0, 'peso': self.pesos['VIA_ADMINISTRACION']}
        
        score_total += score_via * self.pesos['VIA_ADMINISTRACION']
        
        # 3. PRINCIPIO ACTIVO (10% - MENOS CRÍTICO, MÁS FLEXIBLE)
        # PRINCIPIO ACTIVO como BONUS (no afecta score base)
        if medicamento_origen['PRINCIPIO ACTIVO'] == medicamento_candidato['PRINCIPIO ACTIVO']:
            bonus_principio = 0.15  # 15% bonus si coincide exactamente
        elif any(palabra in medicamento_candidato['PRINCIPIO ACTIVO'] for palabra in medicamento_origen['PRINCIPIO ACTIVO'].split() if len(palabra) > 3):
            bonus_principio = 0.10  # 10% bonus si coincide parcialmente
        else:
            bonus_principio = 0.0   # Sin bonus pero NO penaliza

        score_total += bonus_principio  # Se suma ENCIMA del 100%
        
        # 4. FORMA FARMACÉUTICA (10% - IMPORTANTE)
        if medicamento_origen['FORMA FARMACÉUTICA'] == medicamento_candidato['FORMA FARMACÉUTICA']:
            score_forma = 1.0
            detalle['FORMA'] = {'coincide': True, 'score': 1.0, 'peso': self.pesos['FORMA_FARMACEUTICA']}
        else:
            score_forma = 0.5  # Penalización menor para forma farmacéutica
            detalle['FORMA'] = {'coincide': False, 'score': 0.5, 'peso': self.pesos['FORMA_FARMACEUTICA']}
        
        score_total += score_forma * self.pesos['FORMA_FARMACEUTICA']
        
        # 5. SIMILITUD DE CANTIDAD (5% - IMPORTANTE)
        cantidad_origen = medicamento_origen.get('CANTIDAD', 0)
        cantidad_candidato = medicamento_candidato.get('CANTIDAD', 0)
        
        if cantidad_origen > 0 and cantidad_candidato > 0:
            ratio = min(cantidad_origen, cantidad_candidato) / max(cantidad_origen, cantidad_candidato)
            if ratio < 0.5:
                score_cantidad = 0.1
            elif ratio < 0.8:
                score_cantidad = 0.4
            else:
                score_cantidad = ratio
        else:
            score_cantidad = 0.3
        
        detalle['CANTIDAD'] = {
            'origen': cantidad_origen, 
            'candidato': cantidad_candidato, 
            'score': score_cantidad, 
            'peso': self.pesos['CANTIDAD_SIMILITUD']
        }
        
        score_total += score_cantidad * self.pesos['CANTIDAD_SIMILITUD']
        
        return score_total, detalle
    
    def generar_motivo_recomendacion(self, detalle_scoring: dict, medicamento_origen: dict, medicamento_candidato: dict) -> str:
        """
        Genera explicación del motivo de recomendación.
        
        Args:
            detalle_scoring (dict): Detalle del scoring calculado
            medicamento_origen (dict): Medicamento de consulta
            medicamento_candidato (dict): Medicamento recomendado
            
        Returns:
            str: Explicación del motivo de recomendación
        """
        motivos = []
        
        # Motivos críticos
        if detalle_scoring['ATC']['coincide']:
            motivos.append(f"✅ Mismo ATC ({medicamento_origen['ATC']})")
        
        if detalle_scoring['VIA']['coincide']:
            motivos.append(f"✅ Misma vía ({medicamento_origen['VÍA ADMINISTRACIÓN']})")
        
        if 'PRINCIPIO' in detalle_scoring and detalle_scoring['PRINCIPIO']['coincide']:
            motivos.append(f"✅ Mismo principio activo ({medicamento_origen['PRINCIPIO ACTIVO']})")
        elif 'PRINCIPIO_BONUS' in detalle_scoring:
            if detalle_scoring['PRINCIPIO_BONUS']['bonus'] > 0.10:
                motivos.append(f"✅ Principio activo compatible (bonus +{detalle_scoring['PRINCIPIO_BONUS']['bonus']:.0%})")
            elif detalle_scoring['PRINCIPIO_BONUS']['bonus'] > 0:
                motivos.append(f"⚠️ Principio activo parcialmente compatible (bonus +{detalle_scoring['PRINCIPIO_BONUS']['bonus']:.0%})")
        
        # Motivos importantes
        if detalle_scoring['FORMA']['coincide']:
            motivos.append(f"✅ Misma forma farmacéutica ({medicamento_origen['FORMA FARMACÉUTICA']})")
        else:
            motivos.append(f"⚠️ Forma diferente: {medicamento_origen['FORMA FARMACÉUTICA']} → {medicamento_candidato['FORMA FARMACÉUTICA']}")
        
        # Motivo de cantidad
        if detalle_scoring['CANTIDAD']['score'] > 0.8:
            motivos.append(f"✅ Cantidad similar ({detalle_scoring['CANTIDAD']['origen']} vs {detalle_scoring['CANTIDAD']['candidato']})")
        elif detalle_scoring['CANTIDAD']['score'] > 0.5:
            motivos.append(f"⚠️ Cantidad diferente ({detalle_scoring['CANTIDAD']['origen']} vs {detalle_scoring['CANTIDAD']['candidato']})")
        
        return " | ".join(motivos)
    
    def recomendar_homologos(self, cum_origen: str, n_recomendaciones: int = 5, score_minimo: float = 0.85) -> dict:
        """
        Encuentra medicamentos homólogos para un CUM dado.
        
        Args:
            cum_origen (str): CUM del medicamento a homologar
            n_recomendaciones (int): Número máximo de recomendaciones
            score_minimo (float): Score mínimo para considerar como homólogo válido
            
        Returns:
            dict: Resultado con recomendaciones o razón de no homologación
            
        Examples:
            >>> resultado = sistema.recomendar_homologos('10045-1', n_recomendaciones=5)
            >>> print(f"Encontradas {len(resultado['recomendaciones'])} recomendaciones")
        """
        print(f"🔍 BUSCANDO HOMÓLOGOS PARA CUM: {cum_origen}")
        print("=" * 60)
        
        # 1. Obtener información del medicamento origen
        medicamento_origen = self.modelo.obtener_info_medicamento(cum_origen)
        
        if medicamento_origen is None:
            return {
                'cum_origen': cum_origen,
                'encontrado': False,
                'error': 'CUM no encontrado en el dataset',
                'recomendaciones': []
            }
        
        print(f"📋 MEDICAMENTO ORIGEN:")
        print(f"   🔸 Producto: {medicamento_origen['PRODUCTO']}")
        print(f"   🔸 ATC: {medicamento_origen['ATC']}")
        print(f"   🔸 Vía de administración: {medicamento_origen['VÍA ADMINISTRACIÓN']}")
        print(f"   🔸 Principio activo: {medicamento_origen['PRINCIPIO ACTIVO']}")
        print(f"   🔸 Forma farmacéutica: {medicamento_origen['FORMA FARMACÉUTICA']}")
        print(f"   🔸 Cantidad: {medicamento_origen['CANTIDAD']} {medicamento_origen['UNIDAD MEDIDA']}")
        print(f"   🔸 Válido: {'✅ Sí' if medicamento_origen['VALIDO'] == 1 else '❌ No'}")
        
        # 2. Verificar si tiene combinación ATC+VÍA válida
        combo_origen = f"{medicamento_origen['ATC_label']}_{medicamento_origen['VÍA ADMINISTRACIÓN_label']}"
        
        if combo_origen not in self.modelo.combo_clusters:
            return {
                'cum_origen': cum_origen,
                'encontrado': False,
                'error': f'No hay medicamentos válidos con la combinación ATC+VÍA: {medicamento_origen["ATC"]} + {medicamento_origen["VÍA ADMINISTRACIÓN"]}',
                'recomendaciones': []
            }
        
        # 3. Buscar candidatos con la misma combinación ATC+VÍA
        candidatos = (self.modelo.df_validos
                     .filter(
                         (pl.col('ATC') == medicamento_origen['ATC']) &
                         (pl.col('VÍA ADMINISTRACIÓN') == medicamento_origen['VÍA ADMINISTRACIÓN']) &
                         #(pl.col('PRINCIPIO ACTIVO') == medicamento_origen['PRINCIPIO ACTIVO']) &
                         (pl.col('CUM') != cum_origen)  # Excluir el medicamento origen
                     ))
        
        print(f"🎯 CANDIDATOS CON MISMA COMBINACIÓN CRÍTICA: {candidatos.height}")
        
        if candidatos.height == 0:
            return {
                'cum_origen': cum_origen,
                'encontrado': False,
                'error': f'No hay otros medicamentos válidos con ATC: {medicamento_origen["ATC"]}, VÍA: {medicamento_origen["VÍA ADMINISTRACIÓN"]}, PRINCIPIO: {medicamento_origen["PRINCIPIO ACTIVO"]}',
                'recomendaciones': []
            }
        
        # 4. Calcular scores de similitud para cada candidato
        recomendaciones = []
        
        for i in range(candidatos.height):
            candidato = candidatos.row(i, named=True)
            candidato_dict = dict(candidato)
            
            # Calcular score
            score, detalle = self.calcular_score_similitud(medicamento_origen, candidato_dict)
            
            # Solo incluir si supera el score mínimo
            if score >= score_minimo:
                motivo = self.generar_motivo_recomendacion(detalle, medicamento_origen, candidato_dict)
                
                recomendaciones.append({
                    'cum': candidato_dict['CUM'],
                    'producto': candidato_dict['PRODUCTO'],
                    'atc': candidato_dict['ATC'],
                    'via': candidato_dict['VÍA ADMINISTRACIÓN'],
                    'principio_activo': candidato_dict['PRINCIPIO ACTIVO'],
                    'forma_farmaceutica': candidato_dict['FORMA FARMACÉUTICA'],
                    'cantidad': candidato_dict['CANTIDAD'],
                    'unidad': candidato_dict['UNIDAD MEDIDA'],
                    'score_similitud': round(score, 4),
                    'motivo': motivo,
                    'detalle_scoring': detalle
                })
        
        # 5. Ordenar por score y limitar número
        recomendaciones.sort(key=lambda x: x['score_similitud'], reverse=True)
        recomendaciones = recomendaciones[:n_recomendaciones]
        
        print(f"✅ RECOMENDACIONES ENCONTRADAS: {len(recomendaciones)}")
        
        if len(recomendaciones) == 0:
            return {
                'cum_origen': cum_origen,
                'encontrado': False,
                'error': f'Ningún medicamento alcanzó el score mínimo de {score_minimo}',
                'candidatos_evaluados': candidatos.height,
                'recomendaciones': []
            }
        
        return {
            'cum_origen': cum_origen,
            'medicamento_origen': {
                'producto': medicamento_origen['PRODUCTO'],
                'atc': medicamento_origen['ATC'],
                'via': medicamento_origen['VÍA ADMINISTRACIÓN'],
                'principio_activo': medicamento_origen['PRINCIPIO ACTIVO'],
                'es_valido': medicamento_origen['VALIDO'] == 1
            },
            'encontrado': True,
            'candidatos_evaluados': candidatos.height,
            'recomendaciones': recomendaciones,
            'parametros': {
                'score_minimo': score_minimo,
                'n_recomendaciones': n_recomendaciones
            }
        }
    
    def mostrar_resultado_bonito(self, resultado):
        """Muestra el resultado de recomendaciones en formato bonito y legible."""
        print("\n" + "="*80)
        print(f"🔍 BÚSQUEDA DE HOMÓLOGOS PARA CUM: {resultado['cum_origen']}")
        print("="*80)
        
        if not resultado['encontrado']:
            print(f"❌ ERROR: {resultado['error']}")
            return
        
        # Medicamento origen
        origen = resultado['medicamento_origen']
        print(f"📋 MEDICAMENTO ORIGEN:")
        print(f"   🔸 Producto: {origen['producto']}")
        print(f"   🔸 ATC: {origen['atc']}")
        print(f"   🔸 Vía de administración: {origen['via']}")
        print(f"   🔸 Principio activo: {origen['principio_activo']}")
        print(f"   🔸 Válido: {'✅ Sí' if origen['es_valido'] else '❌ No'}")
        
        print(f"\n🎯 CANDIDATOS EVALUADOS: {resultado['candidatos_evaluados']}")
        print(f"✅ RECOMENDACIONES ENCONTRADAS: {len(resultado['recomendaciones'])}")
        
        # Recomendaciones
        for i, rec in enumerate(resultado['recomendaciones'], 1):
            print(f"\n{i}. 📦 {rec['producto']}")
            print(f"   🆔 CUM: {rec['cum']}")
            print(f"   💊 ATC: {rec['atc']}")
            print(f"   🚪 Vía: {rec['via']}")
            print(f"   🧪 Principio activo: {rec['principio_activo']}")
            print(f"   💊 Forma farmacéutica: {rec['forma_farmaceutica']}")
            print(f"   📏 Cantidad: {rec['cantidad']} {rec['unidad']}")
            print(f"   ⭐ Score: {rec['score_similitud']:.1%}")
            print(f"   📝 Motivo: {rec['motivo']}")
            if i < len(resultado['recomendaciones']):
                print("   " + "-"*60)

## Instanciación del Sistema de Recomendación

Creación del sistema de recomendación utilizando el modelo de clustering entrenado para facilitar búsquedas de homólogos.

In [6]:
# Crear sistema de recomendación
print("🎯 CREANDO SISTEMA DE RECOMENDACIÓN DE HOMÓLOGOS")
print("=" * 60)

sistema_homologacion = SistemaRecomendacionHomologos(modelo_homologacion)

print("✅ Sistema de recomendación listo")
print("\n🔥 EJEMPLO DE USO:")
print("resultado = sistema_homologacion.recomendar_homologos('15100-1', n_recomendaciones=5)")
print("sistema_homologacion.mostrar_resultado_bonito(resultado)")

🎯 CREANDO SISTEMA DE RECOMENDACIÓN DE HOMÓLOGOS
✅ Sistema de recomendación listo

🔥 EJEMPLO DE USO:
resultado = sistema_homologacion.recomendar_homologos('15100-1', n_recomendaciones=5)
sistema_homologacion.mostrar_resultado_bonito(resultado)


## Exportación del Modelo Entrenado

Guardado del modelo completo (clustering + sistema de recomendación) para uso posterior en otros scripts o aplicaciones. El modelo se guarda en formato pickle en la carpeta `./models/` con el nombre `iamed.pkl`.

In [10]:
## Guardado del Modelo Entrenado

# Crear directorio de modelos si no existe
import os
import pickle

models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)

# Guardar el modelo completo
model_path = os.path.join(models_dir, 'iamed.pkl')

print("💾 GUARDANDO MODELO ENTRENADO")
print("=" * 40)
print(f"📁 Directorio: {models_dir}")
print(f"📂 Archivo: iamed.pkl")

# Crear un diccionario con todo lo necesario para usar el modelo
modelo_completo = {
    'modelo_clustering': modelo_homologacion,
    'sistema_recomendacion': sistema_homologacion,
    'timestamp': pd.Timestamp.now(),
    'descripcion': 'Sistema completo de homologación de medicamentos con clustering jerárquico',
    'version': '1.0.0'
}

# Guardar usando pickle
with open(model_path, 'wb') as f:
    pickle.dump(modelo_completo, f)

print(f"✅ Modelo guardado exitosamente en: {model_path}")
print(f"📊 Tamaño del archivo: {os.path.getsize(model_path) / (1024*1024):.2f} MB")
print("\n🔥 MODO DE USO:")
print("import pickle")
print("with open('./models/iamed.pkl', 'rb') as f:")
print("    modelo_cargado = pickle.load(f)")
print("sistema = modelo_cargado['sistema_recomendacion']")
print("resultado = sistema.recomendar_homologos('CUM-AQUI')")

💾 GUARDANDO MODELO ENTRENADO
📁 Directorio: ../models
📂 Archivo: iamed.pkl
✅ Modelo guardado exitosamente en: ../models\iamed.pkl
📊 Tamaño del archivo: 88.19 MB

🔥 MODO DE USO:
import pickle
with open('./models/iamed.pkl', 'rb') as f:
    modelo_cargado = pickle.load(f)
sistema = modelo_cargado['sistema_recomendacion']
resultado = sistema.recomendar_homologos('CUM-AQUI')


In [13]:
## Ejemplo de Carga del Modelo

def cargar_modelo_iamed(ruta_modelo='../models/iamed.pkl'):
    """
    Función para cargar el modelo de homologación de medicamentos.
    
    Args:
        ruta_modelo (str): Ruta al archivo del modelo guardado
        
    Returns:
        dict: Diccionario con el modelo y sistema de recomendación
        
    Example:
        >>> modelo = cargar_modelo_iamed()
        >>> sistema = modelo['sistema_recomendacion']
        >>> resultado = sistema.recomendar_homologos('2203-1')
    """
    import pickle
    import os
    
    if not os.path.exists(ruta_modelo):
        raise FileNotFoundError(f"El modelo no existe en la ruta: {ruta_modelo}")
    
    print(f"📂 Cargando modelo desde: {ruta_modelo}")
    
    with open(ruta_modelo, 'rb') as f:
        modelo_completo = pickle.load(f)
    
    print(f"✅ Modelo cargado exitosamente")
    print(f"📊 Versión: {modelo_completo.get('version', 'N/A')}")
    print(f"📅 Fecha entrenamiento: {modelo_completo.get('timestamp', 'N/A')}")
    print(f"📝 Descripción: {modelo_completo.get('descripcion', 'N/A')}")
    
    return modelo_completo

# Ejemplo de uso
print("🧪 EJEMPLO DE CARGA DEL MODELO:")
print("=" * 35)
print("modelo = cargar_modelo_iamed()")
print("sistema = modelo['sistema_recomendacion']")
print("resultado = sistema.recomendar_homologos('2203-1', n_recomendaciones=3)")
print("sistema.mostrar_resultado_bonito(resultado)")

🧪 EJEMPLO DE CARGA DEL MODELO:
modelo = cargar_modelo_iamed()
sistema = modelo['sistema_recomendacion']
resultado = sistema.recomendar_homologos('2203-1', n_recomendaciones=3)
sistema.mostrar_resultado_bonito(resultado)


## Prueba Práctica del Sistema

Ejemplo de búsqueda de medicamentos homólogos utilizando un CUM específico para demostrar el funcionamiento del sistema.

In [17]:
modelo = cargar_modelo_iamed()
sistema = modelo['sistema_recomendacion']
resultado = sistema.recomendar_homologos('2203-1', n_recomendaciones=3)
sistema.mostrar_resultado_bonito(resultado)

📂 Cargando modelo desde: ../models/iamed.pkl
✅ Modelo cargado exitosamente
📊 Versión: 1.0.0
📅 Fecha entrenamiento: 2025-06-23 13:05:10.726264
📝 Descripción: Sistema completo de homologación de medicamentos con clustering jerárquico
🔍 BUSCANDO HOMÓLOGOS PARA CUM: 2203-1
📋 MEDICAMENTO ORIGEN:
   🔸 Producto: ACETAMINOFEN JARABE X 150 MG / 5 ML
   🔸 ATC: N02BE01
   🔸 Vía de administración: ORAL
   🔸 Principio activo: ACETAMINOFEN POLVO
   🔸 Forma farmacéutica: JARABE
   🔸 Cantidad: 3.0 g
   🔸 Válido: ❌ No
🎯 CANDIDATOS CON MISMA COMBINACIÓN CRÍTICA: 532
✅ RECOMENDACIONES ENCONTRADAS: 3

🔍 BÚSQUEDA DE HOMÓLOGOS PARA CUM: 2203-1
📋 MEDICAMENTO ORIGEN:
   🔸 Producto: ACETAMINOFEN JARABE X 150 MG / 5 ML
   🔸 ATC: N02BE01
   🔸 Vía de administración: ORAL
   🔸 Principio activo: ACETAMINOFEN POLVO
   🔸 Válido: ❌ No

🎯 CANDIDATOS EVALUADOS: 532
✅ RECOMENDACIONES ENCONTRADAS: 3

1. 📦 ACETAMINOFEN JARABE
   🆔 CUM: 19929516-3
   💊 ATC: N02BE01
   🚪 Vía: ORAL
   🧪 Principio activo: ACETAMINOFEN
   💊 For

In [11]:
resultado = sistema_homologacion.recomendar_homologos('2203-1', n_recomendaciones=5)
sistema_homologacion.mostrar_resultado_bonito(resultado)

🔍 BUSCANDO HOMÓLOGOS PARA CUM: 2203-1
📋 MEDICAMENTO ORIGEN:
   🔸 Producto: ACETAMINOFEN JARABE X 150 MG / 5 ML
   🔸 ATC: N02BE01
   🔸 Vía de administración: ORAL
   🔸 Principio activo: ACETAMINOFEN POLVO
   🔸 Forma farmacéutica: JARABE
   🔸 Cantidad: 3.0 g
   🔸 Válido: ❌ No
🎯 CANDIDATOS CON MISMA COMBINACIÓN CRÍTICA: 532
✅ RECOMENDACIONES ENCONTRADAS: 5

🔍 BÚSQUEDA DE HOMÓLOGOS PARA CUM: 2203-1
📋 MEDICAMENTO ORIGEN:
   🔸 Producto: ACETAMINOFEN JARABE X 150 MG / 5 ML
   🔸 ATC: N02BE01
   🔸 Vía de administración: ORAL
   🔸 Principio activo: ACETAMINOFEN POLVO
   🔸 Válido: ❌ No

🎯 CANDIDATOS EVALUADOS: 532
✅ RECOMENDACIONES ENCONTRADAS: 5

1. 📦 ACETAMINOFEN JARABE
   🆔 CUM: 19929516-3
   💊 ATC: N02BE01
   🚪 Vía: ORAL
   🧪 Principio activo: ACETAMINOFEN
   💊 Forma farmacéutica: JARABE
   📏 Cantidad: 3.0 g
   ⭐ Score: 110.0%
   📝 Motivo: ✅ Mismo ATC (N02BE01) | ✅ Misma vía (ORAL) | ✅ Misma forma farmacéutica (JARABE) | ✅ Cantidad similar (3.0 vs 3.0)
   -------------------------------------